In [ ]:
%load_ext autoreload
%autoreload 2
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from sklearn.decomposition import NMF
%matplotlib inline
from utils import *
from vca import *

## Simulation Training Data
```python
wave_list = [750, 760, 800, 850, 900, 925]
wave_list = [750, 760, 800, 850, 900, 910, 920, 930, 940, 950]
```

In [ ]:
wave_list = [750, 760, 800, 850, 900, 925]
abs_coeff, legend = {}, ["HbO2", "Hb", "Cholesterol", "Background"]
wave_abs = np.load('./data/hbo2hbchpr_57.npy')

f = loadmat("./data/unmix.mat")
X, Y = f['x'], f['y']

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = (idx, wave_abs[idx])
coeffs = np.vstack([abs_coeff[wave][1] for wave in wave_list])
weights_plot(array = coeffs[:, 0:], wave_list = wave_list, legend = legend, figsize = (9, 6), xticks = wave_list)

## Linear Unmixing

In [ ]:
depth = 25
hbhbo2fat = np.copy(coeffs)[:, 0:3]
sim_data = np.array([np.array(loadmat(f"./data/hb_hbo2_fat_29_{depth}/PA_Image_{wave}.mat")['Image_PA']) for wave in wave_list])

In [ ]:
unmixed = run_linear_unmixing(sim_data.copy(), hbhbo2fat)
plot_comps_2d(unmixed, wave_list, hbhbo2fat, clim = [None]*3, xticks = wave_list)

In [ ]:
plot_3d_multiple(Y*1000, X*1000, unmixed, title = legend[:3], cmap = 'jet', clim = [None]*3, order = [0, 1, 2])

In [ ]:
plt.figure(figsize = (20, 12))
for i in range(sim_data.shape[0]):
    plt.subplot(2, sim_data.shape[0] // 2, i+1)
    plt.imshow(sim_data[i], cmap = "jet")
    plt.title(f'{wave_list[i]} nm')
    plt.colorbar()
    plt.clim([0, 0.005])

## NMF

In [ ]:
nmf_model = NMF(n_components = 3)
nmf_model.fit(sim_data.copy().reshape(len(wave_list), -1).T)

In [ ]:
nmf_comps = nmf_model.fit_transform(sim_data.copy().reshape(len(wave_list), -1).T).reshape((396, 101, 3))
plot_comps_2d(nmf_comps, wave_list, nmf_model.components_.T, order = [0, 1, 2], clim = [None]*3)

In [ ]:
plot_3d_multiple(Y*1000, X*1000, nmf_comps, title = legend[:3], cmap = 'jet', clim = [[0, np.max(nmf_comps)]]*3, order = [0, 1, 2])

## VCA

In [ ]:
ae, ind, yp = vca(sim_data.copy().reshape(len(wave_list), -1), 3)
vca_comps = np.matmul(np.linalg.pinv(ae), yp).reshape((3, 396, 101)).transpose((1, 2, 0))
plot_comps_2d(vca_comps, wave_list, ae, order = [2, 0, 1], clim = [None, None, None])

In [ ]:
plot_3d_multiple(Y*1000, X*1000, vca_comps, title = legend[:3], cmap = 'jet', clim = [None]*3, order = [2, 0, 1])

## FastICA

```python
from scipy import linalg
XW = sim_data.reshape((10, 396*101))
X_mean = XW.mean(axis = -1)
XW -= X_mean[:, np.newaxis]
U, D = linalg.svd(XW, full_matrices = False, check_finite = False)[:2]
U *= np.sign(U[0])
K = (U / D).T[:3]
XW = np.dot(K, XW)
XW *= np.sqrt(396*101)
XW = XW.reshape((3, 396, 101)).transpose((1, 2, 0))
```

In [ ]:
for i in range(20):
    print(f"Random State: {i}")
    maps, wts, _ = run_ica(sim_data.copy(), wave_list, 3, i)
    plot_comps_2d(maps, wave_list, wts, figsize = (10, 3), order = [0, 1, 2])

In [ ]:
order = [2, 1, 0]
maps, wts, model = run_ica(sim_data.copy(), wave_list, 3, 0)
plot_comps_2d(maps, wave_list, np.linalg.pinv(model.components_[order, :]), xticks = wave_list, clim = [None]*3, order = order)

In [ ]:
plot_3d_multiple(Y*1000, X*1000, maps, title = legend[:3], cmap = 'jet', clim = [[0, np.max(maps)]]*3, order = order)

In [ ]:
depth = 40
hbhbo2fat = np.copy(coeffs)[:, 0:3]
sim_data_test = np.array([np.array(loadmat(f"../data/hb_hbo2_fat_29_{depth}/PA_Image_{wave}.mat")['Image_PA']) for wave in wave_list])
sim_data_test = sim_data_test.reshape((10, 396 * 101)).T
sim_data_test -= model.mean_
test_comps = np.matmul(model.components_, sim_data_test.T).reshape((3, 396, 101)).transpose((1, 2, 0))
plot_comps_2d(test_comps, wave_list, np.linalg.pinv(model.components_), order = order, clim = [None]*3)

In [ ]:
plot_3d_multiple(Y*1000, X*1000, test_comps, title = legend[:3], cmap = 'jet', clim = [None]*3, order = order)

## 10 Wavelengths Experimental Results

```python
f = loadmat("./expdata/All Animal Results to Date/CONTROL MICE/03.31 Exp 2 (10 WV)/ALL_FRAMES_MOTIONREMOVED.mat")

f = loadmat("./expdata/All Animal Results to Date/CONTROL MICE/03.31 Exp 7 (10 WV)/SUMMARY_DATA.mat")

f = loadmat("./expdata/All Animal Results to Date/DKO MICE/01.20 Exp 19 (10 WV)/SUMMARY_FRAMES.mat")

f = loadmat("./expdata/All Animal Results to Date/EX VIVO SKIN SAMPLES/01.28 Exp 23/SUMMARY_FRAMES.mat")
```

In [ ]:
f = loadmat("./expdata/All Animal Results to Date/DKO MICE/01.20 Exp 19 (10 WV)/SUMMARY_FRAMES.mat")
exp_wave_list = [int(key[-3:]) for key in f.keys() if key[:3] == 'all']
exp_img = normalize(np.array([np.mean(f[f'all{wave}'], axis = 2) for wave in exp_wave_list]).transpose((1, 2, 0))[60:210, 50:170, :])
exp_img = exp_img.transpose((2, 0, 1))

plt.figure(figsize = (24, 7))
for i, wave in enumerate(exp_wave_list):
    plt.subplot(2, 5, i+1)
    plt.imshow(exp_img[i], cmap = "hot")
    plt.title(f"{wave} nm")
    plt.colorbar()
    plt.clim([0, np.max(exp_img)])
plt.show()

### Linear Unmixing

In [ ]:
exp_unmixed = run_linear_unmixing(exp_img.copy(), hbhbo2fat)
plot_comps_2d(exp_unmixed, wave_list, hbhbo2fat, "Linear Unmixing", (18, 4), order = [0, 1, 2], xticks = wave_list, clim = [None]*3)

### FastICA

In [ ]:
exp_test_data = exp_img.copy().transpose((1, 2, 0)).reshape((-1, len(wave_list)))
exp_test_data -= model.mean_
exp_test_comps = np.matmul(model.components_, exp_test_data.T).T.reshape((exp_img.shape[1], exp_img.shape[2], 3))
plot_comps_2d(exp_test_comps, wave_list, np.linalg.pinv(model.components_), "ICA", (18, 4), order = [2, 1, 0], invert_sign = None, clim = [None, None, [0, 6]], xticks = wave_list)

### NMF

In [ ]:
exp_test_data = exp_img.copy().transpose((1, 2, 0)).reshape((-1, len(wave_list)))
nmf_test_comps = nmf_model.transform(exp_test_data).reshape((exp_img.shape[1], exp_img.shape[2], 3))
plot_comps_2d(nmf_test_comps, wave_list, nmf_model.components_.T, order = [0, 1, 2], clim = [None]*3)

### VCA

In [ ]:
vca_test_comps = np.matmul(np.linalg.pinv(ae), exp_test_data.copy().T).reshape((3, exp_img.shape[1], exp_img.shape[2])).transpose((1, 2, 0))
plot_comps_2d(vca_test_comps, wave_list, ae, order = [2, 0, 1], clim = [None, None, [0, 60]])

### 6 Wavelengths Experimental Results

```python
f = loadmat("./expdata/All Animal Results to Date/EX VIVO SKIN SAMPLES/01.28 Exp 22/SUMMARY_FRAMES.mat")
exp_wave_list = [int(key[-3:]) for key in f.keys() if key[:3] == 'all']
exp_img = normalize(np.array([np.mean(f[f'all{wave}'], axis = 2) for wave in exp_wave_list]).transpose((1, 2, 0))[50:150, :, :])
exp_img = exp_img.transpose((2, 0, 1))
```

In [ ]:
f = loadmat("expdata/All Animal Results to Date/APOE MICE/01.28 Exp 16/SUMMARY_FRAMES.mat")
wave_data = f[f'all{wave_list[0]}']

In [ ]:
f = loadmat("expdata/All Animal Results to Date/APOE MICE/01.28 Exp 16/SUMMARY_FRAMES.mat")
exp_wave_list = [int(key[-3:]) for key in f.keys() if key[:3] == 'all']
exp_img = normalize(np.array([np.mean(f[f'all{wave}'], axis = 2) for wave in exp_wave_list]).transpose((1, 2, 0))[55:155, 60:, :])
exp_img = exp_img.transpose((2, 0, 1))

plt.figure(figsize = (24, 7))
for i, wave in enumerate(exp_wave_list):
    plt.subplot(2, 3, i+1)
    plt.imshow(exp_img[i], cmap = "hot")
    plt.title(f"{wave} nm")
    plt.colorbar()
    plt.clim([0, np.max(exp_img)])
plt.show()

### Linear Unmixing

In [ ]:
exp_unmixed = run_linear_unmixing(exp_img.copy(), hbhbo2fat)
plot_comps_2d(exp_unmixed, wave_list, hbhbo2fat, "Linear Unmixing", (18, 4), order = [0, 1, 2], xticks = wave_list, clim = [None]*3)

### FastICA

In [ ]:
exp_test_data = exp_img.copy().transpose((1, 2, 0)).reshape((-1, len(wave_list)))
exp_test_data -= model.mean_
exp_test_comps = np.matmul(model.components_, exp_test_data.T).T.reshape((exp_img.shape[1], exp_img.shape[2], 3))
plot_comps_2d(exp_test_comps, wave_list, np.linalg.pinv(model.components_), "ICA", (18, 4), order = order, clim = [None, None, [0, 0.5]], xticks = wave_list)

### NMF

In [ ]:
exp_test_data = exp_img.copy().transpose((1, 2, 0)).reshape((-1, len(wave_list)))
nmf_test_comps = nmf_model.transform(exp_test_data).reshape((exp_img.shape[1], exp_img.shape[2], 3))
plot_comps_2d(nmf_test_comps, wave_list, nmf_model.components_.T, order = [0, 1, 2], clim = [None]*3)

### VCA

In [ ]:
vca_test_comps = np.matmul(np.linalg.pinv(ae), exp_test_data.copy().T).reshape((3, exp_img.shape[1], exp_img.shape[2])).transpose((1, 2, 0))
plot_comps_2d(vca_test_comps, wave_list, ae, order = [2, 0, 1], clim = [None, None, [0, 60]])